In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [1]:
#to confirm its using the blazing fast GPU
#!nvidia-smi 

Thu Jul 17 17:18:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-16GB           On  |   00000000:00:1B.0 Off |                    0 |
| N/A   42C    P0             39W /  300W |       1MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import sys
sys.path.append("../src")
from dataset import ComplaintDataset

In [2]:
df = pd.read_csv("downloads/complaints_train.csv")
df_clean = df[["narrative", "product"]].dropna().astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df_clean["product"])

texts = df_clean["narrative"].tolist()
labels = le.transform(df_clean["product"].tolist())

In [ ]:
#!pip install torch --quiet
#!pip install transformers --quiet


In [6]:
# STEP 3: Tokenizer, Dataset, DataLoader
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

dataset = ComplaintDataset(texts, labels, tokenizer)
dataloader = DataLoader(dataset, batch_size=32)


In [7]:
print(ComplaintDataset)

<class 'dataset.ComplaintDataset'>


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("models/best_model")
model.to("cuda")
model.eval()


In [2]:
!pip install torch transformers scikit-learn --quiet

In [10]:
!python ../src/train.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
Epoch 1 - Validation: 100%|███████████████████| 813/813 [00:21<00:00, 37.97it/s]
Epoch 1 | Train Loss: 0.4241 | Val Loss: 0.3512
✅ Best model saved.
Epoch 2 - Validation: 100%|███████████████████| 813/813 [00:21<00:00, 37.77it/s]
Epoch 2 | Train Loss: 0.3059 | Val Loss: 0.3412
✅ Best model saved.
Epoch 3 - Validation: 100%|███████████████████| 813/813 [00:21<00:00, 37.74it/s]
Epoch 3 | Train Loss: 0.2377 | Val Loss: 0.3447


In [ ]:
##############################################

In [12]:
import sys
sys.path.append("../src")

from model import DistilBERTWithCustomHead

In [13]:
#Step 1: Load model + tokenizer
from transformers import AutoTokenizer
from model import DistilBERTWithCustomHead
import torch

MODEL_PATH = "../models/best_model"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = DistilBERTWithCustomHead(num_labels=5)  # change if not 5
model.load_state_dict(torch.load(f"{MODEL_PATH}/model.pt"))
model.to("cuda")
model.eval()


DistilBERTWithCustomHead(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1

In [14]:
#Step 2: Prepare test data
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from dataset import ComplaintDataset
from torch.utils.data import DataLoader

df = pd.read_csv("../notebooks/downloads/complaints_test.csv")
df_clean = df[["narrative", "product"]].dropna().astype(str)

le = LabelEncoder()
le.fit(df_clean["product"])  # same label order as train

texts = df_clean["narrative"].tolist()
y_true = le.transform(df_clean["product"].tolist())

test_dataset = ComplaintDataset(texts, y_true, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32)


In [15]:
# Step 3: Run inference
all_preds = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to("cuda")
        attention_mask = batch["attention_mask"].to("cuda")
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())


In [19]:
import pandas as pd
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

# 🔥 Step 3: Print metrics
print("\n--- NLP Model Evaluation ---")
print("Precision:", precision_score(y_true, all_preds, average="macro"))
print("Recall:", recall_score(y_true, all_preds, average="macro"))
print("F1 Score:", f1_score(y_true, all_preds, average="macro"))
print("\n", classification_report(y_true, all_preds, target_names=le.classes_))



--- NLP Model Evaluation ---
Precision: 0.8742638246205822
Recall: 0.8297976031686766
F1 Score: 0.849505932617285

                      precision    recall  f1-score   support

        credit_card       0.79      0.83      0.81      3114
   credit_reporting       0.90      0.96      0.93     18235
    debt_collection       0.86      0.73      0.79      4630
mortgages_and_loans       0.91      0.79      0.85      3798
     retail_banking       0.91      0.84      0.87      2707

           accuracy                           0.89     32484
          macro avg       0.87      0.83      0.85     32484
       weighted avg       0.89      0.89      0.88     32484

